# Test of readout code for LTC2324 ADC Chip

The overlay contains both the readout module for the ADC and a dummy module to generate simulated data.  The interface to the LTC2324 is through PMOD with the following pinout
- PMODA[0]: external trigger input, active HI
- PMODA[1]: cvt output, active LOW
- PMODA[2]: sck output
- PMODA[3]: clkout input
- PMODA[4]: sdo serial data input

The dummy ADC connections are on PMODB with pins corresponding to the same pins on PMODA
- PMODB[1]: dummy_cvt input
- PMODB[2]: dummy_sck input
- PMODB[3]: dummy_clkout output
- PMODB[4]: dummy_sdo serial data output

If PMODB pins 1-4 are jumpered to the same pins on on PMOD, then the module should read back the data written to the dummy_ADC register defined below.

In [11]:
# Because of a bug/feature of the Pynq library, this is necessary when loading updated
# versions of the same design.
from pynq import PL
PL.reset()

In [12]:
# load the overlay
from pynq import Overlay
pynq = Overlay("mani_readout.bit")

In [13]:
help(pynq)

Help on Overlay in module pynq.overlay:

<pynq.overlay.Overlay object>
    Default documentation for overlay mani_readout.bit. The following
    attributes are available on this overlay:
    
    IP Blocks
    ----------
    control              : pynq.lib.axigpio.AxiGPIO
    data                 : pynq.lib.axigpio.AxiGPIO
    state                : pynq.lib.axigpio.AxiGPIO
    dummy_ADC            : pynq.lib.axigpio.AxiGPIO
    timing               : pynq.lib.axigpio.AxiGPIO
    processing_system7_0 : pynq.overlay.DefaultIP
    
    Hierarchies
    -----------
    LTC2324_dummy_0      : pynq.overlay.DefaultHierarchy
    LTC2324_read_0       : pynq.overlay.DefaultHierarchy
    
    Interrupts
    ----------
    None
    
    GPIO Outputs
    ------------
    None
    
    Memories
    ------------
    PSDDR                : Memory



In [14]:
# Define the communication registes
# Interface to LTC2324 readout module
timing = pynq.timing   # timing control word
                       # timing[7:4] = number of clock cyccles for cvt low before first valid bit
                       # timing[3:0] = number of clock cycles for SCK_LO and SCK_HI
control = pynq.control # control word
                       # control[0] = arm
                       # control[1] = soft trigger
        
state = pynq.state     # state of readount.  There will be data ready whan state==4
data = pynq.data       # data register
# Interface to dummy data generation module
dummy_ADC = pynq.dummy_ADC  # Dummy data for the data simulation


In [50]:
# Initial setup
timing.write(0,0x86)       # 40 ns for everything
dummy_ADC.write(0,0xAA55)  # dummy ADC data

In [51]:
print(state.read(0))

4


In [52]:
control.write(0,1)  # arm
control.write(0,2)  # trigger

In [53]:
print('%x'%data.read(0))

aa55


In [37]:
while True:
    control.write(0,0)
    control.write(0,1)
    control.write(0,2)


KeyboardInterrupt

